# Pipeline Optimization

<img src="img/pipeline.jpg">

# Search over Pipeline-land...

<img src="img/caruna15-ml-pipeline-research.png">

# What do we want?

* Strong Anytime Performance
* Strong Final Performance
* Effective Use of Parallel Resources
* Scalability
* Robustness & Flexibility
* Simplicity
* Computational efficiency

See [A. Biedenkapp, *BOHB Robust and Efficient Hyperparameter Optimization at Scale*](https://www.automl.org/blog_bohb/)

# Approaches

### Bayesian Optimization

### Genetic Programming

### Meta-learning

### Multi-armed bandits

### Reinforcement Learning for Neural Architecture Search

# Bayesian Optimization

Can we use the same approach that we use for Hyper-parameter Optimization to search for the best ML pipeline?

### Problem
Parameter space becomes huge and we need to handle *conditional parameters*, parameters that are only relevant when others are set in a specific way (e.g. `svm__kernel` only relevant if `classifier=svm`).
This has been refered to as the CASH problem: *Combined Algorithm Selection and Hyper-parameter optimization*.



# Genetic Programming

GP is a evolutionary computation technique for automatically constructing computer programs.

GP evolves computer programs, traditionally represented in memory as tree structures. Every tree node has an operator function and every terminal node has an operand, making mathematical expressions easy to evolve and evaluate (e.g. for symbolic regression).

[Algorithm](http://www.genetic-programming.com/gpanimatedtutorial.html):

  1. Randomly create an initial population (generation 0)
  2. Iteratively perform the following sub-steps (called a generation):
    1. Execute each program in the population and ascertain its fitness
    2. Select programs from the population with a proba based on fitness to participate in the genetic operations in C.
    3. Create new programs for the population by applying the following genetic operations with specified probabilities:
      1. Reproduction: Copy the selected individual program to the new population.
      2. Crossover: Create new offspring program(s) for the new population by recombining randomly chosen parts from two selected programs.
      3. Mutation: Create one new offspring program for the new population by randomly mutating a randomly chosen part of one selected program.
  

# Mutation

<img src="img/gp-mutation.png" >

# Cross-over

<img src="img/gp-cross-over.png" >


# Meta-Learning

Learn from the performance of ML piplines from **different** datasets to predict which ML pipelines will likely do well on the dataset at hand.

Can we used to create a seed set for other techniques (e.g. [auto-sklearn](https://www.automl.org/automl/auto-sklearn/) uses Meta-Learning and Bayesian Optimization). 

Meta-learning model: target is performance and runtime; features are characteristics of the dataset, pipeline, and problem formulation.

Training data: [OpenML](https://www.openml.org/) .

Portfolio selection for diverse initial solutions .

<div  style="float: right;" >
<img src="img/automl-portfolio-selection.gif" width=300>
<div style="text-align: right">Source: A. Biedenkapp, "BOHB Robust and Efficient Hyperparameter Optimization at Scale."</div>
</div>
 

# Multi-armed Bandits

### Idea
Sample randomly N pipelines, each of them represents an *arm*. We want to select the arm with the highest yield (ie best performance) by spending the least *budget* in total. 

### [SuccessiveHalving](http://proceedings.mlr.press/v51/jamieson16.pdf)
HS (or its extension *Hyperband*) is a bandit technique.
Budget can be iterations (for anytime algorithms), data points (!), or features (!).

<img src="img/map-sh.gif" width=500 >
<div style="text-align: right">Source: A. Biedenkapp, "BOHB Robust and Efficient Hyperparameter Optimization at Scale."</div>

Can be seen as an application of early-stopping to pipeline optimization.

# Reinforcement Learning for Neural Architecture Search

The architecture of a Neural Network can also be seen as a DAG.

All of the above methods have been proposed for NAS but recent research showed excellent results with Reinforcement Learning

<img src="img/zoph-nas.png" width=500 >
<div style="text-align: right">Source: B. Zoph, Q. Le (2017), "Neural Architecture Search with Reinforcement Learning"</div>

# Reinforcement Learning for NAS cont'

### Controller Network

Recurrent neural network that can generate neural architectures by emitting a string/token-sequence that encodes the architecture (layers, cells, hyper-parameters). The image below describes how the tokens for n-th layer are generated:

<img src="img/zoph-rnn.png" width=500 >
<div style="text-align: right">Source: B. Zoph, Q. Le (2017), "Neural Architecture Search with Reinforcement Learning"</div>

Once all actions are performed (ie. tokens emitted), the network can be trained and we receive a reward, the performance metric on the validation data and the controller is updated. 

# NAS results

Zoph & Le's RL approach obtained competitive performance on the CIFAR-10 and Penn Treebank benchmarks but used vast computational resources to do so (800 GPUs for 3-4 weeks)

<img src="img/zoph-cifar10-results.png" width=500 >
<div style="text-align: right">Source: B. Zoph, Q. Le (2017), "Neural Architecture Search with Reinforcement Learning"</div>

# Case-study: [TPOT](http://epistasislab.github.io/tpot/)

AML tool that optimizes machine learning pipelines using *genetic programming*.

<img src="img/tpot-logo.jpg" width="200" style="float: right;">

Developed by Randal S. Olson and others at the University of Pennsylvania.

<img src="img/tpot-ml-pipeline.png"  width="600" style="float: left;">


In [ ]:
from tpot import TPOTRegressor
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

housing = load_boston()
X_train, X_test, y_train, y_test = train_test_split(housing.data, housing.target,
                                                    train_size=0.75, test_size=0.25)

In [ ]:
tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

In [ ]:
tpot.export('tpot_boston_pipeline.py')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator

# NOTE: Make sure that the class is labeled 'target' in the data file
tpot_data = load_boston()
features = tpot_data.data
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data.target, random_state=None)

# Average CV score on the training set was:-9.779522448419902
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=AdaBoostRegressor(learning_rate=0.5, loss="linear", n_estimators=100)),
    StackingEstimator(estimator=GradientBoostingRegressor(alpha=0.95, learning_rate=0.1, loss="ls", max_depth=4, max_features=0.55, min_samples_leaf=17, min_samples_split=2, n_estimators=100, subsample=0.8500000000000001)),
    AdaBoostRegressor(learning_rate=0.5, loss="exponential", n_estimators=100)
)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)


# State-of-the-art

Many state-of-the-art approaches to Pipeline Optimization are hybrids of the above methods:

  * Meta-learning for initialization and portfolio selection
  * Bayesian Optimization for fine tuning once good candidates were identified
  
<img src="img/automl-state-of-the-art.png" width=500 >
<div style="text-align: right">Source: A. Biedenkapp, "BOHB Robust and Efficient Hyperparameter Optimization at Scale."</div>

# State-of-the-art cont'

### Results

<img src="img/automl-results.png" width=500 >
<div style="text-align: right">Source: A. Biedenkapp, "BOHB Robust and Efficient Hyperparameter Optimization at Scale."</div>

### Academic Competitions

  * [PAKDD 2019 Data Competition](https://www.4paradigm.com/competition/pakdd2019)
  * [PAKDD 2018 Data Competition](https://www.4paradigm.com/competition/pakdd2018)
  * [IJCNN 2015-2016 AutoML challenge](https://competitions.codalab.org/competitions/2321)